In [ ]:
%tensorflow_version 2.x

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
#dataset reading
# tfds.disable_progress_bar()
(train_ds, test_ds), ds_info = tfds.load(
    'food101', 
    as_supervised=True,
    with_info=True,
    split=['train[:90%]', 'train[90%:]'],
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
#visualization
some_samples = [x[0] for x in iter(train_ds.take(32))]

fig = plt.figure(figsize=(16, 8))
for j in range(len(some_samples)):
    ax = fig.add_subplot(4, 8, j+1)
    ax.imshow(some_samples[j])
    plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
#dataset preparation, image resizing
INP_SIZE = 224
NUM_EPOCHS = 10
BATCH_SIZE = 64

def prepare(img, label):
    img = tf.cast(img, tf.float32)/255.
    return tf.image.resize(img, (INP_SIZE, INP_SIZE)), label

train_ds = train_ds.shuffle(buffer_size=1000)
train_ds = train_ds.map(prepare)
train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=True)

test_ds = test_ds.shuffle(buffer_size=1000)
test_ds = test_ds.map(prepare)
test_ds = test_ds.batch(128, drop_remainder=True)

In [ ]:
print(list(train_ds.as_numpy_iterator())[0])

In [ ]:
# # Conv + BatchNorm + ReLU
# class CBR(tf.keras.layers.Layer):
    
#     def __init__(self, filters, kernel_size, **kwargs):
#         super().__init__()        
#         self.conv = tf.keras.layers.Conv2D(filters, kernel_size, **kwargs)
#         self.bn = tf.keras.layers.BatchNormalization()
        
#     def call(self, inp, training=False):
#         out = self.conv(inp)
#         out = self.bn(out, training)
#         out = tf.nn.relu(out)        
#         return out

In [ ]:
# Loading MobileNetV2 for finetunig and using for food101
EXP_NAME = 'transfer'
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(INP_SIZE, INP_SIZE, 3),
    include_top=False,
    weights='imagenet',
)
# base_model.trainable = True # Fine-tuning весов предобученной модели
base_model.trainable = False # Заморозка весов предобученной модели

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(1280),
    tf.keras.layers.Dense(101, activation='sigmoid'),
])

In [ ]:
#preparation for learning
LEARNING_RATE = 0.001
optimizer = tf.keras.optimizers.RMSprop(lr=LEARNING_RATE)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+EXP_NAME, 
    write_graph=False, update_freq=100, profile_batch=0)

In [ ]:
model.summary()

In [ ]:
labels

In [ ]:
%%time

history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=test_ds,
    callbacks=[tensorboard_callback])